In [23]:
import requests
import pandas as pd
from geopy.distance import geodesic
import networkx as nx
import os
import json
from dotenv import load_dotenv

In [24]:
# Carregar variáveis do arquivo .env
load_dotenv('/Users/enzooliveira/Pessoal/VS CODE/RoutesWithA*/.env')

# Acessar as variáveis
API_KEY = os.getenv('API_KEY')
BASE_URL = os.getenv('BASE_URL')


# Parâmetros da requisição
params = {
    "output": "json",
    "countrycode": "BR",  # Alterar conforme necessário
    "maxresults": 1000,  # Número de resultados
    "compact": True,  # Retorno compacto
    "verbose": False,
    "key": API_KEY
}

# Enviando a requisição
response = requests.get(BASE_URL, params=params)

# Processando a resposta
if response.status_code == 200:
    charging_points = response.json()
    print("Número de pontos encontrados:", len(charging_points))

    # Criar uma lista para armazenar os dados de cada ponto de carregamento
    data = []
    for point in charging_points:
        # Extrair as informações
        name = point.get('AddressInfo', {}).get('Title', 'Desconhecido')
        latitude = point.get('AddressInfo', {}).get('Latitude')
        longitude = point.get('AddressInfo', {}).get('Longitude')
        
        # Inicializar o maior PowerKW como 0
        max_power_kw = 0
        
        # Extrair os conectores (se existirem) e pegar o maior PowerKW
        connectors = point.get('Connections', [])
        for connector in connectors:
            max_power_kw = max(max_power_kw, connector.get('PowerKW', 0))  # Pega o maior PowerKW
        
        # Adicionar os dados à lista
        data.append({
            'Nome': name,
            'Localização': f"{latitude}, {longitude}",
            'PowerKW': max_power_kw
        })

    # Criar um DataFrame com os dados
    df = pd.DataFrame(data)

    # Exibir o DataFrame
    print(df)
else:
    print(f"Erro ao acessar API: {response.status_code}")

Número de pontos encontrados: 397
                                             Nome  \
0    VAPT Cajazeiras | Grupo Barbosa Home Center    
1             VAPT Campina Grande | Club do Trato   
2                       VAPT Natal | Grupo Centry   
3                 VAPT João Pessoa | Grupo Centry   
4         VAPT Parnamirim | Sacolão Maria Lacerda   
..                                            ...   
392                                      BR Mania   
393             Av. Epitácio Pessoa, 2900 - Lagoa   
394             Av. Epitácio Pessoa, 2900 - Lagoa   
395                            Recife, Pernambuco   
396              Petrobras Future Station Service   

                        Localização  PowerKW  
0         -6.90123354, -38.56828009     40.0  
1          -7.2213914, -35.87546424     40.0  
2         -5.81160289, -35.22604487     40.0  
3          -7.1244542, -34.87325013     40.0  
4         -5.88960119, -35.20911729     40.0  
..                              ...      ...  


In [25]:
df

,Nome,Localização,PowerKW
0,VAPT Cajazeiras | Grupo Barbosa Home Center,"-6.90123354, -38.56828009",40.0
1,VAPT Campina Grande | Club do Trato,"-7.2213914, -35.87546424",40.0
2,VAPT Natal | Grupo Centry,"-5.81160289, -35.22604487",40.0
3,VAPT João Pessoa | Grupo Centry,"-7.1244542, -34.87325013",40.0
4,VAPT Parnamirim | Sacolão Maria Lacerda,"-5.88960119, -35.20911729",40.0
...,...,...,...
392,BR Mania,"-22.97294, -43.20274",50.0
393,"Av. Epitácio Pessoa, 2900 - Lagoa","-22.97294, -43.20274",0.0
394,"Av. Epitácio Pessoa, 2900 - Lagoa","-22.97294, -43.20274",0.0
395,"Recife, Pernambuco","-14.235004, -51.92527999999999",0.0


In [4]:
#Filtrar linhas onde 'PowerKW' é igual a 0
filtered_df = df[df['PowerKW'] == 0]

filtered_df

,Nome,Localização,PowerKW
9,Posto Luz da Lua - InWatts,"-25.46926759912884, -49.35278832712896",0.0
136,Rodovia Governador Leonel de Moura Brizola,"-29.692107611323813, -51.70509274127477",0.0
262,Byd Carmais,"-3.738428409647497, -38.50797373048607",0.0
275,BYD Itavema,"-23.006053601412404, -43.432111951119026",0.0
281,Rodoviária de João Neiva,"-19.755933400729162, -40.380491082223216",0.0
326,Fashion Outlet Novo Hamburgo,"-29.663541425765796, -51.14242583068132",0.0
342,Teresópolis Shopping,"-22.41143127179805, -42.965655579837176",0.0
364,Rua Maria de Freitas 639,"-21.120077, -44.170421",0.0
367,Plaza Shopping Niterói,"-22.88759, -43.124492",0.0
390,BR Mania,"-22.97294, -43.20274",0.0


In [5]:
# Remover as linhas onde 'PowerKW' é igual a 0 do DataFrame original
df = df[df['PowerKW'] != 0]

df

,Nome,Localização,PowerKW
0,VAPT Cajazeiras | Grupo Barbosa Home Center,"-6.90123354, -38.56828009",40.0
1,VAPT Campina Grande | Club do Trato,"-7.2213914, -35.87546424",40.0
2,VAPT Natal | Grupo Centry,"-5.81160289, -35.22604487",40.0
3,VAPT João Pessoa | Grupo Centry,"-7.1244542, -34.87325013",40.0
4,VAPT Parnamirim | Sacolão Maria Lacerda,"-5.88960119, -35.20911729",40.0
...,...,...,...
386,EDP | Graal Topázio (Plug&GO),"-22.614671702946467, -47.356908583240454",175.0
387,Estação Posto Ilhabela,"-27.578138960692, -48.510338084656",50.0
388,Floripa Shopping,"-27.534450652309026, -48.509559003948254",5.0
389,Posto BR Mania,"-22.97294, -43.20274",50.0


In [6]:
#Filtrar linhas onde 'PowerKW' é igual a 0
filtered_df = df[df['PowerKW'] == 0]

filtered_df

,Nome,Localização,PowerKW


In [7]:
# Função para calcular o tempo de carregamento baseado na distância e PowerKW
def calcular_tempo_carregamento(distancia, power_kw):
    # Tempo de carregamento em horas, baseado na distância e na potência do carregador
    if power_kw == 0:
        return 0  # Se o PowerKW for 0, não há carregamento
    tempo_carregamento = distancia / power_kw  # O tempo de carregamento é inversamente proporcional ao PowerKW
    return tempo_carregamento  # Tempo em horas

# Função para adicionar o tempo de carregamento nas arestas, para todos os vértices conectados, exceto o ponto final (Salvador)
def adicionar_tempo_carregamento(G, df, ponto_final):
    print("Calculando o tempo de carregamento baseado na distância e PowerKW...")

    for u, v, atributos in G.edges(data=True):
        # Ignorar o ponto final (Salvador) e considerar todos os outros nós
        if u != ponto_final and v != ponto_final:
            # Obter o PowerKW dos nós u e v
            power_kw_u = df[df['Nome'] == u]['PowerKW'].values
            power_kw_v = df[df['Nome'] == v]['PowerKW'].values

            # Calcular o tempo de carregamento para a aresta u-v
            if power_kw_u.size > 0 and power_kw_v.size > 0:
                power_kw_u_value = power_kw_u[0]
                power_kw_v_value = power_kw_v[0]

                distancia = atributos['distance']
                tempo_carregamento_u = calcular_tempo_carregamento(distancia, power_kw_u_value)
                tempo_carregamento_v = calcular_tempo_carregamento(distancia, power_kw_v_value)
                
                # A média do tempo de carregamento entre os dois nós
                tempo_carregamento_medio = (tempo_carregamento_u + tempo_carregamento_v) / 2
                atributos['charging_time'] = tempo_carregamento_medio  # Tempo de carregamento em horas
                print(f"Tempo de carregamento de {u} para {v}: {tempo_carregamento_medio:.2f} horas")
            else:
                print(f"Erro ao acessar PowerKW para {u} ou {v}")

In [8]:
# Função para calcular distância geodésica entre dois pontos
def calcular_distancia_geodesica(lat1, lon1, lat2, lon2):
    ponto1 = (lat1, lon1)
    ponto2 = (lat2, lon2)
    return geodesic(ponto1, ponto2).km  # Distância em km

# Função para estimar o tempo de deslocamento com base em uma velocidade média
def calcular_tempo_estimado(distancia):
    velocidade_media_kmh = 100  # Velocidade média estimada em km/h
    return (distancia / velocidade_media_kmh) * 60  

# Função para calcular o tempo de carregamento baseado na distância e PowerKW
def calcular_tempo_carregamento(distancia, power_kw):
    tempo_carregamento = (distancia / power_kw) * 60  # Tempo em minutos
    return tempo_carregamento

# Função para construir o grafo considerando a autonomia
def construir_grafo(df, autonomia_km, pasta, ponto_inicial, ponto_final):
    G = nx.DiGraph()  # Grafo direcionado

    # Iterando sobre todos os pontos de carregamento no DataFrame
    for i, ponto1 in df.iterrows():
        for j, ponto2 in df.iterrows():
            if i != j:
                nome1 = ponto1['Nome']
                nome2 = ponto2['Nome']
                lat1, lon1 = map(float, ponto1['Localização'].split(','))
                lat2, lon2 = map(float, ponto2['Localização'].split(','))
                
                # Calculando a distância geodésica entre os pontos
                distancia = calcular_distancia_geodesica(lat1, lon1, lat2, lon2)
                
                # Se a distância for menor ou igual à autonomia, adiciona a aresta
                if distancia <= autonomia_km:
                    tempo = calcular_tempo_estimado(distancia)
                    power_kw = ponto2['PowerKW']
                    tempo_carregamento = calcular_tempo_carregamento(distancia, power_kw)
                    
                    G.add_edge(nome1, nome2, weight_distancia=distancia, weight_tempo=tempo, weight_carregamento=tempo_carregamento)

    # Adicionar o ponto inicial ao grafo
    G.add_node('inicial')
    for i, ponto in df.iterrows():
        nome = ponto['Nome']
        lat1, lon1 = ponto_inicial
        lat2, lon2 = map(float, ponto['Localização'].split(','))
        distancia = calcular_distancia_geodesica(lat1, lon1, lat2, lon2)
        if distancia <= autonomia_km:
            tempo = calcular_tempo_estimado(distancia)
            tempo_carregamento = calcular_tempo_carregamento(distancia, ponto['PowerKW'])
            G.add_edge('inicial', nome, weight_distancia=distancia, weight_tempo=tempo, weight_carregamento=tempo_carregamento)

    # Adicionar o ponto final ao grafo
    G.add_node('final')
    for i, ponto in df.iterrows():
        nome = ponto['Nome']
        lat1, lon1 = map(float, ponto['Localização'].split(','))
        lat2, lon2 = ponto_final
        distancia = calcular_distancia_geodesica(lat1, lon1, lat2, lon2)
        if distancia <= autonomia_km:
            tempo = calcular_tempo_estimado(distancia)
            G.add_edge(nome, 'final', weight_distancia=distancia, weight_tempo=tempo, weight_carregamento=0)  # Sem tempo de carregamento

    # Salvar o grafo assim que ele for gerado
    salvar_grafo(G, autonomia_km, pasta)

    return G

# Função para salvar o grafo em diferentes formatos
def salvar_grafo(grafo, autonomia, pasta):
    # Criar a pasta se ela não existir
    if not os.path.exists(pasta):
        os.makedirs(pasta)

    nome_base = os.path.join(pasta, f'grafo_{autonomia}_km')

    # Salvando em .gpickle
    try:
        nx.write_gpickle(grafo, f'{nome_base}.gpickle')
        print(f"Grafo salvo em {nome_base}.gpickle")
    except AttributeError as e:
        print(f"Erro ao salvar em .gpickle: {e}")

    # Salvando em JSON
    try:
        with open(f'{nome_base}.json', 'w') as f:
            json.dump(nx.node_link_data(grafo), f)
        print(f"Grafo salvo em {nome_base}.json")
    except Exception as e:
        print(f"Erro ao salvar em .json: {e}")

    # Salvando em CSV
    try:
        arestas = []
        for u, v, data in grafo.edges(data=True):
            arestas.append({
                'origem': u,
                'destino': v,
                'distancia': data['weight_distancia'],
                'tempo': data['weight_tempo'],
                'carregamento': data['weight_carregamento']
            })
        df_arestas = pd.DataFrame(arestas)
        df_arestas.to_csv(f'{nome_base}.csv', index=False)
        print(f"Grafo salvo em {nome_base}.csv")
    except Exception as e:
        print(f"Erro ao salvar em .csv: {e}")

# Coordenadas de pontos iniciais e finais
sjc_coords = (-23.1896, -45.9009)  # Aproximadamente
salvador_coords = (-12.9714, -38.5014)  # Aproximadamente

# Criando os grafos apenas para autonomias de 400 km e 500 km
grafos = {}
autonomias = [400, 500]  # Apenas autonomias de 400 km e 500 km

for autonomia in autonomias:
    pasta = f'{autonomia}km'  # Pasta específica para cada autonomia
    grafos[autonomia] = construir_grafo(df, autonomia, pasta, sjc_coords, salvador_coords)

Erro ao salvar em .gpickle: module 'networkx' has no attribute 'write_gpickle'
Grafo salvo em 400km/grafo_400_km.json
Grafo salvo em 400km/grafo_400_km.csv


/opt/anaconda3/envs/tf-metal/lib/python3.10/site-packages/networkx/readwrite/json_graph/node_link.py:142: FutureWarning: 
The default value will be `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_data(G, edges="links") to preserve current behavior, or
  nx.node_link_data(G, edges="edges") for forward compatibility.
  warnings.warn(


Erro ao salvar em .gpickle: module 'networkx' has no attribute 'write_gpickle'
Grafo salvo em 500km/grafo_500_km.json
Grafo salvo em 500km/grafo_500_km.csv


In [9]:
# Carregar o CSV do grafo de 400 km
df_400_km = pd.read_csv('400km/grafo_400_km.csv')

# Mostrar todos os vértices e arestas com os parâmetros
print("Grafo de 400 km:")
print("\nVértices:")
vertices_400 = set(df_400_km['origem']).union(set(df_400_km['destino']))
print(vertices_400)

print("\nArestas:")
for _, row in df_400_km.iterrows():
    origem = row['origem']
    destino = row['destino']
    distancia = row['distancia']
    tempo = row['tempo']
    carregamento = row['carregamento']
    #print(f"Origem: {origem}, Destino: {destino}, Distância: {distancia:.2f} km, Tempo: {tempo:.2f} min, Carregamento: {carregamento:.2f} min")

Grafo de 400 km:

Vértices:
{'Eletroposto DCC Energy', 'Câmara dos Deputados', 'Planeta Charge', 'Eurobike Special BYD', 'Casarão Goiano', 'BR Super Carga - Bela Vista Empresarial', 'VAPT Natal | Grupo Centry', 'Audi Center Fortaleza', 'VAPT João Pessoa | Intense Body Center', 'BR Super Carga - Cafe do Rancho', 'Eletroposto Eletroposto Volvo Carga Rápida', 'Posto Flecha', 'Cluster Go Electric - Cajamar', 'Administração Regional de Samambaia', 'Eletrograal Alemão', 'Posto Amigão de Itaipava', 'Posto Graal Parada Norte', 'Lanchonete Lili Doces - BR Super Carga', 'Planeta Charge - Jean da Picanha', 'Hotel Pousada Itajuipe', 'Shell Recharge - Posto Shell São Gualter', 'BR Super Carga - Pagelar', 'VAPT Natal | Sacolão Maranguape', 'Office 661', 'EDP Cachoeiro de Itapemirim', 'Rodovia Washington Luís', 'Trevo Posto da Mata', 'DCC Energy - Íbis Hotel', 'DCC Energy - Reserva Open Mall', 'Volvo - Salgueiro Plaza Hotel', 'Posto Ipiranga Peruzzo', 'Posto Bizungao III', 'AGL Eletrônicos do Brasil'

In [10]:
# Carregar o CSV do grafo de 500 km
df_500_km = pd.read_csv('500km/grafo_500_km.csv')

# Mostrar todos os vértices e arestas com os parâmetros
print("Grafo de 500 km:")
print("\nVértices:")
vertices_500 = set(df_500_km['origem']).union(set(df_500_km['destino']))
print(vertices_500)

print("\nArestas:")
for _, row in df_500_km.iterrows():
    origem = row['origem']
    destino = row['destino']
    distancia = row['distancia']
    tempo = row['tempo']
    carregamento = row['carregamento']
    #print(f"Origem: {origem}, Destino: {destino}, Distância: {distancia:.2f} km, Tempo: {tempo:.2f} min, Carregamento: {carregamento:.2f} min")

Grafo de 500 km:

Vértices:
{'Eletroposto DCC Energy', 'Câmara dos Deputados', 'Planeta Charge', 'Eurobike Special BYD', 'Casarão Goiano', 'BR Super Carga - Bela Vista Empresarial', 'VAPT Natal | Grupo Centry', 'Audi Center Fortaleza', 'VAPT João Pessoa | Intense Body Center', 'BR Super Carga - Cafe do Rancho', 'Eletroposto Eletroposto Volvo Carga Rápida', 'Posto Flecha', 'Cluster Go Electric - Cajamar', 'Administração Regional de Samambaia', 'Eletrograal Alemão', 'Posto Amigão de Itaipava', 'Posto Graal Parada Norte', 'Lanchonete Lili Doces - BR Super Carga', 'Planeta Charge - Jean da Picanha', 'Hotel Pousada Itajuipe', 'Shell Recharge - Posto Shell São Gualter', 'BR Super Carga - Pagelar', 'VAPT Natal | Sacolão Maranguape', 'Office 661', 'EDP Cachoeiro de Itapemirim', 'Rodovia Washington Luís', 'Trevo Posto da Mata', 'DCC Energy - Íbis Hotel', 'DCC Energy - Reserva Open Mall', 'Volvo - Salgueiro Plaza Hotel', 'Posto Ipiranga Peruzzo', 'Posto Bizungao III', 'AGL Eletrônicos do Brasil'

In [11]:
def carregar_grafo_csv(arquivo_csv):
    # Lê o CSV e constrói um grafo direcionado
    df = pd.read_csv(arquivo_csv)
    G = nx.DiGraph()
    for _, row in df.iterrows():
        origem = row['origem']
        destino = row['destino']
        distancia = row['distancia']
        tempo = row['tempo']
        carregamento = row['carregamento']
        # Adiciona aresta ao grafo
        G.add_edge(origem, destino, weight_distancia=distancia, weight_tempo=tempo, weight_carregamento=carregamento)
    return G

def verificar_caminho(grafo, nome_grafo):
    try:
        if nx.has_path(grafo, 'inicial', 'final'):
            # Calcula o caminho mais curto baseado na distância
            caminho = nx.shortest_path(grafo, source='inicial', target='final', weight='weight_distancia')
            distancia_total = sum(grafo[origem][destino]['weight_distancia'] for origem, destino in zip(caminho[:-1], caminho[1:]))
            print(f"Caminho no {nome_grafo}: {' -> '.join(map(str, caminho))}")
            print(f"Distância total: {distancia_total:.2f} km")
        else:
            print(f"Não há caminho disponível no {nome_grafo} entre 'inicial' e 'final'.")
    except nx.NetworkXNoPath:
        print(f"Não há caminho disponível no {nome_grafo} entre 'inicial' e 'final'.")
    except Exception as e:
        print(f"Ocorreu um erro no {nome_grafo}: {e}")

# Carregar os grafos a partir dos arquivos CSV
grafo_400_km = carregar_grafo_csv('400km/grafo_400_km.csv')
grafo_500_km = carregar_grafo_csv('500km/grafo_500_km.csv')

# Verificar conectividade nos grafos
print("Verificando conectividade:")
verificar_caminho(grafo_400_km, "grafo de 400 km")
verificar_caminho(grafo_500_km, "grafo de 500 km")

Verificando conectividade:
Caminho no grafo de 400 km: inicial -> Graal Itatiaia -> Posto Senna -> Hotel Pioneiro -> Volvo - Novo Posto Verao -> final
Distância total: 1475.54 km
Caminho no grafo de 500 km: inicial -> Cabana da Mantiqueira -> EDP São Mateus -> Posto Flecha -> final
Distância total: 1456.03 km


In [12]:
# Função para carregar o grafo a partir de um CSV
def carregar_grafo_csv(arquivo_csv):
    df = pd.read_csv(arquivo_csv)
    G = nx.DiGraph()
    for _, row in df.iterrows():
        origem = row['origem']
        destino = row['destino']
        distancia = row['distancia']
        tempo = row['tempo']
        carregamento = row['carregamento']
        # Adicionando as arestas com os pesos no grafo
        G.add_edge(
            origem, 
            destino, 
            weight_tempo= 0.5 * tempo + 0.5 * carregamento,  # Peso total (tempo de deslocamento + carregamento)
            weight_distancia=distancia, 
            weight_deslocamento=tempo, 
            weight_carregamento=carregamento
        )
    return G

# Função para executar o algoritmo A*
def algoritmo_a_estrela(grafo, nome_grafo):
    try:
        # Usar o algoritmo A* do NetworkX
        caminho = nx.astar_path(grafo, source='inicial', target='final', heuristic=lambda u, v: 0, weight='weight_tempo')

        # Calcular os tempos e distâncias totais
        tempo_total = sum(grafo[origem][destino]['weight_tempo'] for origem, destino in zip(caminho[:-1], caminho[1:]))
        tempo_deslocamento = sum(grafo[origem][destino]['weight_deslocamento'] for origem, destino in zip(caminho[:-1], caminho[1:]))
        tempo_recarregando = sum(grafo[origem][destino]['weight_carregamento'] for origem, destino in zip(caminho[:-1], caminho[1:]))
        distancia_total = sum(grafo[origem][destino]['weight_distancia'] for origem, destino in zip(caminho[:-1], caminho[1:]))

        # Exibir os resultados
        print(f"Grafo {nome_grafo}:")
        print(f"Caminho: {' -> '.join(map(str, caminho))}")
        print(f"Tempo total: {tempo_total:.2f} min")
        print(f"Tempo de deslocamento: {tempo_deslocamento:.2f} min")
        print(f"Tempo recarregando: {tempo_recarregando:.2f} min")
        print(f"Distância total: {distancia_total:.2f} km")
        print(f"Tempo total (deslocamento + carregamento): {tempo_total:.2f} min\n")

    except nx.NetworkXNoPath:
        print(f"Não há caminho no grafo {nome_grafo} de 'inicial' para 'final'.")
    except Exception as e:
        print(f"Ocorreu um erro no grafo {nome_grafo}: {e}")

# Carregar os grafos de 400km e 500km a partir dos CSVs
grafo_400_km = carregar_grafo_csv('400km/grafo_400_km.csv')
grafo_500_km = carregar_grafo_csv('500km/grafo_500_km.csv')

# Executar o algoritmo A* para os dois grafos
algoritmo_a_estrela(grafo_400_km, "400km")
algoritmo_a_estrela(grafo_500_km, "500km")

Grafo 400km:
Caminho: inicial -> Audi Center Juiz de Fora -> Audi Center Vitoria -> Trevo Posto da Mata -> Volvo - Novo Posto Verao -> final
Tempo total: 946.65 min
Tempo de deslocamento: 898.11 min
Tempo recarregando: 995.18 min
Distância total: 1496.86 km
Tempo total (deslocamento + carregamento): 946.65 min

Grafo 500km:
Caminho: inicial -> Audi Center Juiz de Fora -> Audi Center Vitoria -> Volvo - Novo Posto Verao -> final
Tempo total: 946.62 min
Tempo de deslocamento: 898.09 min
Tempo recarregando: 995.15 min
Distância total: 1496.82 km
Tempo total (deslocamento + carregamento): 946.62 min



In [13]:
# Função para carregar o grafo a partir de um CSV
def carregar_grafo_csv(arquivo_csv):
    df = pd.read_csv(arquivo_csv)
    G = nx.DiGraph()
    for _, row in df.iterrows():
        origem = row['origem']
        destino = row['destino']
        distancia = row['distancia']
        tempo = row['tempo']
        carregamento = row['carregamento']
        # Calculando o novo peso conforme a fórmula: 0.7 * distância + 0.3 * tempo de carregamento
        peso = 0.7 * distancia + 0.3 * carregamento
        # Adicionando as arestas com os pesos no grafo
        G.add_edge(
            origem, 
            destino, 
            weight=peso,  # Novo peso para o algoritmo A*
            weight_distancia=distancia, 
            weight_deslocamento=tempo, 
            weight_carregamento=carregamento
        )
    return G

# Função para executar o algoritmo A*
def algoritmo_a_estrela(grafo, nome_grafo):
    try:
        # Usar o algoritmo A* do NetworkX
        caminho = nx.astar_path(grafo, source='inicial', target='final', heuristic=lambda u, v: 0, weight='weight')

        # Calcular os tempos e distâncias totais
        tempo_total = sum(grafo[origem][destino]['weight'] for origem, destino in zip(caminho[:-1], caminho[1:]))
        tempo_deslocamento = sum(grafo[origem][destino]['weight_deslocamento'] for origem, destino in zip(caminho[:-1], caminho[1:]))
        tempo_recarregando = sum(grafo[origem][destino]['weight_carregamento'] for origem, destino in zip(caminho[:-1], caminho[1:]))
        distancia_total = sum(grafo[origem][destino]['weight_distancia'] for origem, destino in zip(caminho[:-1], caminho[1:]))

        # Exibir os resultados
        print(f"Grafo {nome_grafo}:")
        print(f"Caminho: {' -> '.join(map(str, caminho))}")
        print(f"Peso total (distância + carregamento): {tempo_total:.2f}")
        print(f"Tempo de deslocamento: {tempo_deslocamento:.2f} min")
        print(f"Tempo recarregando: {tempo_recarregando:.2f} min")
        print(f"Distância total: {distancia_total:.2f} km\n")
        print(f"Tempo total (deslocamento + carregamento): {tempo_total:.2f} min\n")

    except nx.NetworkXNoPath:
        print(f"Não há caminho no grafo {nome_grafo} de 'inicial' para 'final'.")
    except Exception as e:
        print(f"Ocorreu um erro no grafo {nome_grafo}: {e}")

# Carregar os grafos de 400km e 500km a partir dos CSVs
grafo_400_km = carregar_grafo_csv('400km/grafo_400_km.csv')
grafo_500_km = carregar_grafo_csv('500km/grafo_500_km.csv')

# Executar o algoritmo A* para os dois grafos
algoritmo_a_estrela(grafo_400_km, "400km")
algoritmo_a_estrela(grafo_500_km, "500km")

Grafo 400km:
Caminho: inicial -> Audi Center Juiz de Fora -> Audi Center Vitoria -> Trevo Posto da Mata -> Volvo - Novo Posto Verao -> final
Peso total (distância + carregamento): 1346.35
Tempo de deslocamento: 898.11 min
Tempo recarregando: 995.18 min
Distância total: 1496.86 km

Tempo total (deslocamento + carregamento): 1346.35 min

Grafo 500km:
Caminho: inicial -> Audi Center Juiz de Fora -> Audi Center Vitoria -> Volvo - Novo Posto Verao -> final
Peso total (distância + carregamento): 1346.32
Tempo de deslocamento: 898.09 min
Tempo recarregando: 995.15 min
Distância total: 1496.82 km

Tempo total (deslocamento + carregamento): 1346.32 min



In [14]:
# Função para carregar o grafo a partir de um CSV
def carregar_grafo_csv(arquivo_csv):
    df = pd.read_csv(arquivo_csv)
    G = nx.DiGraph()
    for _, row in df.iterrows():
        origem = row['origem']
        destino = row['destino']
        distancia = row['distancia']
        tempo = row['tempo']
        carregamento = row['carregamento']
        # Calculando o novo peso conforme a fórmula: 0.45 * tempo + 0.45 * distância + 0.1 * tempo de carregamento
        peso = 0.45 * tempo + 0.45 * distancia + 0.1 * carregamento
        # Adicionando as arestas com os pesos no grafo
        G.add_edge(
            origem, 
            destino, 
            weight=peso,  # Novo peso para o algoritmo A*
            weight_distancia=distancia, 
            weight_deslocamento=tempo, 
            weight_carregamento=carregamento
        )
    return G

# Função para executar o algoritmo A*
def algoritmo_a_estrela(grafo, nome_grafo):
    try:
        # Usar o algoritmo A* do NetworkX
        caminho = nx.astar_path(grafo, source='inicial', target='final', heuristic=lambda u, v: 0, weight='weight')

        # Calcular os tempos e distâncias totais
        tempo_total = sum(grafo[origem][destino]['weight'] for origem, destino in zip(caminho[:-1], caminho[1:]))
        tempo_deslocamento = sum(grafo[origem][destino]['weight_deslocamento'] for origem, destino in zip(caminho[:-1], caminho[1:]))
        tempo_recarregando = sum(grafo[origem][destino]['weight_carregamento'] for origem, destino in zip(caminho[:-1], caminho[1:]))
        distancia_total = sum(grafo[origem][destino]['weight_distancia'] for origem, destino in zip(caminho[:-1], caminho[1:]))

        # Exibir os resultados
        print(f"Grafo {nome_grafo} (Experimento 3):")
        print(f"Caminho: {' -> '.join(map(str, caminho))}")
        print(f"Peso total (tempo + distância + carregamento): {tempo_total:.2f}")
        print(f"Tempo de deslocamento: {tempo_deslocamento:.2f} min")
        print(f"Tempo recarregando: {tempo_recarregando:.2f} min")
        print(f"Distância total: {distancia_total:.2f} km\n")
        print(f"Tempo total (deslocamento + carregamento): {tempo_total:.2f} min\n")

    except nx.NetworkXNoPath:
        print(f"Não há caminho no grafo {nome_grafo} de 'inicial' para 'final'.")
    except Exception as e:
        print(f"Ocorreu um erro no grafo {nome_grafo}: {e}")

# Carregar os grafos de 400km e 500km a partir dos CSVs
grafo_400_km = carregar_grafo_csv('400km/grafo_400_km.csv')
grafo_500_km = carregar_grafo_csv('500km/grafo_500_km.csv')

# Executar o algoritmo A* para os dois grafos
algoritmo_a_estrela(grafo_400_km, "400km")
algoritmo_a_estrela(grafo_500_km, "500km")

Grafo 400km (Experimento 3):
Caminho: inicial -> Audi Center Juiz de Fora -> Audi Center Vitoria -> Trevo Posto da Mata -> Volvo - Novo Posto Verao -> final
Peso total (tempo + distância + carregamento): 1177.26
Tempo de deslocamento: 898.11 min
Tempo recarregando: 995.18 min
Distância total: 1496.86 km

Tempo total (deslocamento + carregamento): 1177.26 min

Grafo 500km (Experimento 3):
Caminho: inicial -> Audi Center Juiz de Fora -> Audi Center Vitoria -> Volvo - Novo Posto Verao -> final
Peso total (tempo + distância + carregamento): 1177.23
Tempo de deslocamento: 898.09 min
Tempo recarregando: 995.15 min
Distância total: 1496.82 km

Tempo total (deslocamento + carregamento): 1177.23 min

